In [1]:
from openmm.app import *
from openmm import *
from openff.toolkit.topology import *
from openmm.unit import *
import openmmtools as omt
from openmmtools.integrators import *
import os
import numpy as np
import MDAnalysis as mda
import pandas as pd
import matplotlib.pyplot as plt
import parmed 
import re
from sys import stdout
%load_ext autoreload
%autoreload 2

5/2/2023
Aayushi:
I think we need EI 
( PBD 2KX9 for open and 2HWG for close) 
Ph 7.4, 
NaCl 100 mM , 
35 celcius
with PEP bound to the C domain. So the Allostery happens when PEP binds and the structure goes from open to Close state. So I think it would be better to start any network analysis on open.

In [7]:
construct = '6dsw'
ff = 'amber'
ligand = 'dna'
structures = '../structures'
pdb_file = f'../structures/protein_{construct}_pqr.pdb'
pdb = PDBFile(pdb_file)
pdb_mg = PDBFile(f'{structures}/Mg_6dsw.pdb')
pdb_ligand = PDBFile(f'{structures}/{ligand}_6dsw_pqr.pdb')
modeller = Modeller(pdb.topology, pdb.positions)
modeller.add(pdb_mg.topology, pdb_mg.positions)
modeller.add(pdb_ligand.topology, pdb_ligand.positions)


## Make an OpenMM System and minimized structure

In [ ]:

# parameterize akg
'''
molecule = Molecule()
ligand_molecule = molecule.from_file(f'{structures}/substrate/{ligand.upper()}.mol')
ligand_molecule.compute_partial_charges_am1bcc()
from openmmforcefields.generators import GAFFTemplateGenerator
gaff = GAFFTemplateGenerator(molecules=ligand_molecule)
# Create an OpenMM ForceField object with AMBER ff14SB and TIP3P with compatible ions
'''
from openmm.app import ForceField
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
# Register the GAFF template generator
#forcefield.registerTemplateGenerator(gaff.generator)
# You can now parameterize an OpenMM Topology object that contains the specified molecule.
# forcefield will load the appropriate GAFF parameters when needed, and antechamber
# will be used to generate small molecule parameters on the fly.

# solvate
modeller.addSolvent(forcefield, model='tip3p', padding=1.5*nanometer, 
                    ionicStrength=0.1*molar )
# create system object
system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME,nonbondedCutoff=1*nanometer, 
                                 constraints=HBonds)
# define temperature and pressure
# 35C
temperature = 308.15 * kelvin
pressure = 1 * bar
# Add pressure control
system.addForce(MonteCarloBarostat(pressure, temperature))
# create integrator object
integrator = LangevinMiddleIntegrator(temperature, 1/picosecond, 2*femtoseconds)
# create simulation object
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)
simulation.minimizeEnergy()
positions = simulation.context.getState(getPositions=True).getPositions()

with open(f'../openmm/{construct}_system.xml', 'w') as outfile:
    outfile.write(XmlSerializer.serialize(system))
with open(f'../openmm/{construct}_minimized.pdb', 'w') as f:
    PDBFile.writeFile(simulation.topology, positions, f)


##### Save a gromacs topology for future trjconv use - Use a no-constraints version of system to avoid parmed error
parmed_system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME,nonbondedCutoff=1*nanometer, rigidWater=False)
pmd_structure = parmed.openmm.load_topology(simulation.topology, system=parmed_system, xyz=positions)

if os.path.exists(f'../gromacs/'):
    pass
else:
    os.makedirs(f'../gromacs/')
    
pmd_structure.save(f"../gromacs/{construct}_SYSTEM.top", overwrite=True)
pmd_structure.save(f"../gromacs/{construct}_SYSTEM.gro", overwrite=True)

##### 


In [5]:
# For plumed scaled topologies - if you want to do HREMD
import sys
sys.path.append('../../../../my_scripts')
from molecular_dynamics_analysis_tools.useful_functions import parmed_underscore_topology

# get protein atom indices
u = mda.Universe(pdb_file)
protein = u.select_atoms('protein')
atom_indices = [atom.index for atom in protein.atoms]

parmed_underscore_topology(f'../gromacs/{construct}_apo/{construct}_amber_apo_SYSTEM.top', 
                           atom_indices, 
                           f'../gromacs/{construct}_apo/{construct}_amber_apo_system_underscored.top')

# Don't forget to remove the underscores after the atom naming